<a href="https://colab.research.google.com/github/KAVYANSHTYAGI/GAN-is-all-you-need/blob/GAN-for-csv-tabular-data-for-ML-models/GAN_data_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
base_dir = "/content/drive/MyDrive/ransomware_analysis_files/csv_data"

In [5]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import SMOTE

# Load the data
data_path = '/content/drive/MyDrive/ransomware_analysis_files/gan_for_synthetic/benign.csv'

dataset = pd.read_csv(data_path)

dataset = dataset.dropna(subset=['Tag_y'])  # Ensure no NaNs in target variable

X = dataset.drop(['Tag_y', 'filename', 'cryptographic_usage_encryption_algorithms',
                  'complexity_metrics_function_count', 'data_flow_collections_usage',
                  'hardcoded_urls', 'obfuscation_techniques_variable_name_length',
                  'unique_suspicious_strings'], axis=1)

y = dataset['Tag_y']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

def build_generator(latent_dim, data_shape):
    model = Sequential([
        layers.Dense(128, activation='relu', input_dim=latent_dim),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(data_shape, activation='sigmoid')  # Adjust based on data range
    ])
    return model

def build_discriminator(data_shape):
    model = Sequential([
        layers.Dense(128, activation='leaky_relu', input_shape=(data_shape,)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='leaky_relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def build_gan(generator, discriminator):
    discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

def train_gan(gan, generator, discriminator, data, latent_dim, epochs, batch_size):
    for epoch in range(epochs):
        # Generate synthetic examples
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_data = generator.predict(noise)

        # Create training set for the discriminator (real + fake)
        real_data = data[np.random.randint(0, data.shape[0], batch_size)]
        combined_data = np.vstack((real_data, generated_data))
        labels = np.concatenate([np.ones(batch_size), np.zeros(batch_size)])

        # Train discriminator
        d_loss = discriminator.train_on_batch(combined_data, labels)

        # Train generator (via GAN model)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_targets = np.ones(batch_size)  # Labels for the generator
        g_loss = gan.train_on_batch(noise, misleading_targets)

        # Optionally print/log losses and save models
        print(f"Epoch: {epoch+1}, D Loss: {d_loss}, G Loss: {g_loss}")





In [ ]:
latent_dim = 80  # Dimensionality of the noise input
generator = build_generator(latent_dim, X.shape[1])
discriminator = build_discriminator(X.shape[1])
gan = build_gan(generator, discriminator)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train the GAN
train_gan(gan, generator, discriminator, X.values, latent_dim, epochs=10000, batch_size=128)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch: 1, D Loss: [array(0.8155298, dtype=float32), array(0.5625, dtype=float32)], G Loss: [array(0.8155298, dtype=float32), array(0.8155298, dtype=float32), array(0.5625, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch: 2, D Loss: [array(0.78087485, dtype=float32), array(0.53515625, dtype=float32)], G Loss: [array(0.78087485, dtype=float32), array(0.78087485, dtype=float32), array(0.53515625, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


Streaming output truncated to the last 5000 lines.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch: 7501, D Loss: [array(nan, dtype=float32), array(0.4525959, dtype=float32)], G Loss: [array(nan, dtype=float32), array(nan, dtype=float32), array(0.4525959, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Epoch: 7502, D Loss: [array(nan, dtype=float32), array(0.45259857, dtype=float32)], G Loss: [array(nan, dtype=float32), array(nan, dtype=float32), array(0.45259857, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch: 7503, D Loss: [array(nan, dtype=float32), array(0.45259967, dtype=float32)], G Loss: [array(nan, dtype=float32), array(nan, dtype=float32), array(0.45259967, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch: 7504, D Loss: [array(nan, dtype=float32), array(0.45259818, dtype=float32)], G Loss: [array(nan, dtype=float32), array(nan, dtype=float32), array(0.45259818, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch: 7505, D Loss: [array(nan, dtyp

In [ ]:

# Save the generator
generator.save('generator_model_ransomware.keras')

# Save the discriminator
discriminator.save('discriminator_model_ransomware.keras')


In [ ]:
from tensorflow.keras.models import load_model
generator = load_model('generator_model_ransomware.keras')
discriminator = load_model('discriminator_model_ransomware.keras')

In [ ]:
def generate_data(generator, latent_dim, num_samples):
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    generated_data = generator.predict(noise)
    return generated_data

synthetic_data = generate_data(generator, latent_dim, 1000)  # Generate 1000 new samples


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
synthetic_data_scaled = scaler.fit_transform(synthetic_data)

In [ ]:
import numpy as np
import pandas as pd

# Assuming 'synthetic_data' is your generated NumPy array from the GAN
# Apply non-negativity constraint
synthetic_data_non_negative = np.abs(synthetic_data)

# Round to nearest integer and ensure all values are integers
synthetic_data_rounded = np.round(synthetic_data_non_negative).astype(int)

# Convert the adjusted synthetic data into a DataFrame
df_synthetic_data = pd.DataFrame(synthetic_data_rounded, columns=X_train.columns)

# Save the DataFrame to a CSV file
df_synthetic_data.to_csv('syntheticdata.csv', index=False)


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

GAN MODEL WITH NOICE INJECTED

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential

class NoiseInjectionLayer(layers.Layer):
    def call(self, inputs):
        noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=1.0)
        return inputs + noise

def build_generator(latent_dim, data_shape):
    model = Sequential([
        layers.Dense(128, activation='relu', input_dim=latent_dim),
        NoiseInjectionLayer(),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        NoiseInjectionLayer(),
        layers.BatchNormalization(),
        layers.Dense(data_shape, activation='sigmoid')
    ])
    return model

def build_discriminator(data_shape):
    model = Sequential([
        layers.Dense(128, activation='leaky_relu', input_shape=(data_shape,)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='leaky_relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def build_gan(generator, discriminator):
    discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

def train_gan(gan, generator, discriminator, data, latent_dim, epochs, batch_size):
    for epoch in range(epochs):
        # Generate synthetic examples
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_data = generator.predict(noise)

        # Create training set for the discriminator (real + fake)
        real_data = data[np.random.randint(0, data.shape[0], batch_size)]
        combined_data = np.vstack((real_data, generated_data))
        labels = np.concatenate([np.ones(batch_size), np.zeros(batch_size)])

        # Train discriminator
        d_loss = discriminator.train_on_batch(combined_data, labels)

        # Train generator (via GAN model)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_targets = np.ones(batch_size)  # Labels for the generator
        g_loss = gan.train_on_batch(noise, misleading_targets)

        # Optionally print/log losses and save models
        if epoch % 10 == 0:
            print(f"Epoch: {epoch+1}, D Loss: {d_loss}, G Loss: {g_loss}")

    return gan


In [ ]:
# Define parameters
latent_dim = 100
data_shape = 784  # example for flattened 28x28 image
epochs = 100
batch_size = 32
data = np.random.rand(1000, data_shape)  # Example data, replace with actual data

# Build and train GAN
generator = build_generator(latent_dim, data_shape)
discriminator = build_discriminator(data_shape)
gan = build_gan(generator, discriminator)
trained_gan = train_gan(gan, generator, discriminator, data, latent_dim, epochs, batch_size)